## Model training and tuning for the consumer discretionary sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.models.forecasting.sf_auto_arima import AutoARIMA
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_real_estate = df[df['GICS Sector'] == 'Real Estate'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [3]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [4]:
df_real_estate['ticker'].unique()

array(['AMT', 'ARE', 'AVB', 'BXP', 'CBRE', 'CCI', 'CPT', 'CSGP', 'DLR',
       'DOC', 'EQIX', 'EQR', 'ESS', 'EXR', 'FRT', 'HST', 'INVH', 'IRM',
       'KIM', 'MAA', 'O', 'PLD', 'PSA', 'REG', 'SBAC', 'SPG', 'UDR',
       'VICI', 'VTR', 'WELL', 'WY'], dtype=object)

In [5]:
data = {}
static_covs = pd.DataFrame({'ticker': df_real_estate['ticker'].unique()})

for ticker, group in df_real_estate.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [6]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [7]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [8]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for AMT
Model for AMT trained in 0.30 seconds
**************************************************
Training model for ARE
Model for ARE trained in 0.31 seconds
**************************************************
Training model for AVB
Model for AVB trained in 0.29 seconds
**************************************************
Training model for BXP
Model for BXP trained in 0.29 seconds
**************************************************
Training model for CBRE
Model for CBRE trained in 0.29 seconds
**************************************************
Training model for CCI
Model for CCI trained in 0.29 seconds
**************************************************
Training model for CPT
Model for CPT trained in 0.29 seconds
**************************************************
Training model for CSGP
Model for CSGP trained in 0.32 seconds
**************************************************
Training model for DLR
Model for DLR trained in 0.34 seconds
***************************************

In [9]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AMT, MAPE: 7.54, MAE: 16.10
**************************************************
Ticker: ARE, MAPE: 6.45, MAE: 6.06
**************************************************
Ticker: AVB, MAPE: 12.43, MAE: 26.95
**************************************************
Ticker: BXP, MAPE: 9.02, MAE: 6.23
**************************************************
Ticker: CBRE, MAPE: 16.71, MAE: 21.51
**************************************************
Ticker: CCI, MAPE: 3.72, MAE: 3.81
**************************************************
Ticker: CPT, MAPE: 2.04, MAE: 2.33
**************************************************
Ticker: CSGP, MAPE: 4.00, MAE: 3.05
**************************************************
Ticker: DLR, MAPE: 6.81, MAE: 11.68
**************************************************
Ticker: DOC, MAPE: 2.86, MAE: 0.58
**************************************************
Ticker: EQIX, MAPE: 9.43, MAE: 79.27
**************************************************
Ticker: EQR, MAPE: 2.39, MAE: 1.67
*********

In [10]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(8.104098871663822, 11.511084274550187)

### Global model

In [11]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 11.56 seconds
**************************************************


In [12]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for AMT, MAPE: 2.55, MAE: 5.24
**************************************************
Global model for ARE, MAPE: 3.47, MAE: 3.53
**************************************************
Global model for AVB, MAPE: 2.44, MAE: 5.20
**************************************************
Global model for BXP, MAPE: 3.09, MAE: 2.15
**************************************************
Global model for CBRE, MAPE: 3.47, MAE: 4.17
**************************************************
Global model for CCI, MAPE: 3.83, MAE: 3.83
**************************************************
Global model for CPT, MAPE: 2.75, MAE: 3.12
**************************************************
Global model for CSGP, MAPE: 2.96, MAE: 2.26
**************************************************
Global model for DLR, MAPE: 2.40, MAE: 3.80
**************************************************
Global model for DOC, MAPE: 2.23, MAE: 0.45
**************************************************
Global model for EQIX, MAPE: 10.17, MAE: 89.42
*

In [13]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(2.9846395466417084, 5.933527984840756)

## Prophet model

In [14]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for AMT


21:51:37 - cmdstanpy - INFO - Chain [1] start processing
21:51:40 - cmdstanpy - INFO - Chain [1] done processing


Model for AMT trained in 3.13 seconds
**************************************************
Training model for ARE


21:51:40 - cmdstanpy - INFO - Chain [1] start processing
21:51:43 - cmdstanpy - INFO - Chain [1] done processing


Model for ARE trained in 3.05 seconds
**************************************************
Training model for AVB


...

Model for VTR trained in 3.15 seconds
**************************************************
Training model for WELL


21:52:58 - cmdstanpy - INFO - Chain [1] start processing
21:53:01 - cmdstanpy - INFO - Chain [1] done processing


Model for WELL trained in 3.28 seconds
**************************************************
Training model for WY


21:53:01 - cmdstanpy - INFO - Chain [1] start processing
21:53:03 - cmdstanpy - INFO - Chain [1] done processing


Model for WY trained in 2.63 seconds
**************************************************


In [15]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AMT, MAPE: 22.74, MAE: 47.30
**************************************************
Ticker: ARE, MAPE: 17.99, MAE: 19.79
**************************************************
Ticker: AVB, MAPE: 16.36, MAE: 35.24
**************************************************
Ticker: BXP, MAPE: 26.43, MAE: 18.98
**************************************************
Ticker: CBRE, MAPE: 24.62, MAE: 31.31
**************************************************
Ticker: CCI, MAPE: 30.39, MAE: 30.60
**************************************************
Ticker: CPT, MAPE: 29.37, MAE: 33.97
**************************************************
Ticker: CSGP, MAPE: 25.49, MAE: 19.29
**************************************************
Ticker: DLR, MAPE: 8.72, MAE: 14.69
**************************************************
Ticker: DOC, MAPE: 35.15, MAE: 7.11
**************************************************
Ticker: EQIX, MAPE: 8.20, MAE: 68.74
**************************************************
Ticker: EQR, MAPE: 20.08, MAE: 1

In [16]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(20.076493868362807, 24.9759796692196)

## Theta model

In [17]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for AMT
Model for AMT trained in 1.21 seconds
**************************************************
Training model for ARE
Model for ARE trained in 0.69 seconds
**************************************************
Training model for AVB
Model for AVB trained in 0.84 seconds
**************************************************
Training model for BXP
Model for BXP trained in 0.68 seconds
**************************************************
Training model for CBRE
Model for CBRE trained in 0.57 seconds
**************************************************
Training model for CCI
Model for CCI trained in 1.11 seconds
**************************************************
Training model for CPT
Model for CPT trained in 0.79 seconds
**************************************************
Training model for CSGP
Model for CSGP trained in 0.83 seconds
**************************************************
Training model for DLR
Model for DLR trained in 0.91 seconds
***************************************

In [18]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AMT, MAPE: 11.63, MAE: 24.67
**************************************************
Ticker: ARE, MAPE: 12.70, MAE: 11.70
**************************************************
Ticker: AVB, MAPE: 11.00, MAE: 23.83
**************************************************
Ticker: BXP, MAPE: 19.06, MAE: 13.93
**************************************************
Ticker: CBRE, MAPE: 23.90, MAE: 30.65
**************************************************
Ticker: CCI, MAPE: 8.00, MAE: 8.22
**************************************************
Ticker: CPT, MAPE: 13.22, MAE: 15.39
**************************************************
Ticker: CSGP, MAPE: 19.76, MAE: 14.93
**************************************************
Ticker: DLR, MAPE: 12.25, MAE: 20.49
**************************************************
Ticker: DOC, MAPE: 10.22, MAE: 2.12
**************************************************
Ticker: EQIX, MAPE: 16.87, MAE: 147.63
**************************************************
Ticker: EQR, MAPE: 8.60, MAE: 6

In [19]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(12.781682044333834, 19.235720154084596)

## Linear Regression

### Local model

In [20]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for AMT
Model for AMT trained in 0.13 seconds
**************************************************
Training model for ARE
Model for ARE trained in 0.12 seconds
**************************************************
Training model for AVB
Model for AVB trained in 0.12 seconds
**************************************************
Training model for BXP
Model for BXP trained in 0.13 seconds
**************************************************
Training model for CBRE
Model for CBRE trained in 0.12 seconds
**************************************************
Training model for CCI
Model for CCI trained in 0.12 seconds
**************************************************
Training model for CPT
Model for CPT trained in 0.12 seconds
**************************************************
Training model for CSGP
Model for CSGP trained in 0.12 seconds
**************************************************
Training model for DLR
Model for DLR trained in 0.13 seconds
***************************************

In [21]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AMT, MAPE: 2217156.37, MAE: 4746344.64
**************************************************
Ticker: ARE, MAPE: 4.27, MAE: 4.16
**************************************************
Ticker: AVB, MAPE: 1.29, MAE: 2.71
**************************************************
Ticker: BXP, MAPE: 2.40, MAE: 1.66
**************************************************
Ticker: CBRE, MAPE: 3.15, MAE: 3.83
**************************************************
Ticker: CCI, MAPE: 4.39, MAE: 4.42
**************************************************
Ticker: CPT, MAPE: 1.85, MAE: 2.09
**************************************************
Ticker: CSGP, MAPE: 2.70, MAE: 2.03
**************************************************
Ticker: DLR, MAPE: 18.26, MAE: 30.17
**************************************************
Ticker: DOC, MAPE: 1.51, MAE: 0.30
**************************************************
Ticker: EQIX, MAPE: 1.84, MAE: 15.51
**************************************************
Ticker: EQR, MAPE: 1.66, MAE: 1.15
*

In [22]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(72591.88226641317, 154048.00663483568)

### Global model

In [23]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 3.67 seconds
**************************************************


In [24]:
mapes_lr_global = {}
maes_lr_global = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes_lr_global[ticker] = mape_
    maes_lr_global[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for AMT, MAPE: 2.37, MAE: 4.79
**************************************************
Global model for ARE, MAPE: 2.64, MAE: 2.65
**************************************************
Global model for AVB, MAPE: 1.62, MAE: 3.41
**************************************************
Global model for BXP, MAPE: 2.51, MAE: 1.74
**************************************************
Global model for CBRE, MAPE: 2.52, MAE: 3.02
**************************************************
Global model for CCI, MAPE: 2.29, MAE: 2.27
**************************************************
Global model for CPT, MAPE: 1.86, MAE: 2.11
**************************************************
Global model for CSGP, MAPE: 2.37, MAE: 1.82
**************************************************
Global model for DLR, MAPE: 2.21, MAE: 3.53
**************************************************
Global model for DOC, MAPE: 2.30, MAE: 0.46
**************************************************
Global model for EQIX, MAPE: 1.91, MAE: 16.08
**

In [25]:
np.mean(list(mapes_lr_global.values())), np.mean(list(maes_lr_global.values()))

(2.1547877803985362, 2.784613457601903)

## ARIMA

In [26]:
models_arima = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoARIMA()

    model.fit(
        series=train[ticker]['close'],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_arima[ticker] = model

Training model for AMT
Model for AMT trained in 3.60 seconds
**************************************************
Training model for ARE
Model for ARE trained in 2.09 seconds
**************************************************
Training model for AVB
Model for AVB trained in 1.72 seconds
**************************************************
Training model for BXP
Model for BXP trained in 2.50 seconds
**************************************************
Training model for CBRE
Model for CBRE trained in 3.27 seconds
**************************************************
Training model for CCI
Model for CCI trained in 2.83 seconds
**************************************************
Training model for CPT
Model for CPT trained in 1.45 seconds
**************************************************
Training model for CSGP
Model for CSGP trained in 1.20 seconds
**************************************************
Training model for DLR
Model for DLR trained in 1.92 seconds
***************************************

In [27]:
mapes_arima = {}
maes_arima = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_arima[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_arima[ticker] = mape_
    maes_arima[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: AMT, MAPE: 12.52, MAE: 26.50
**************************************************
Ticker: ARE, MAPE: 10.96, MAE: 10.11
**************************************************
Ticker: AVB, MAPE: 11.54, MAE: 24.98
**************************************************
Ticker: BXP, MAPE: 17.02, MAE: 12.50
**************************************************
Ticker: CBRE, MAPE: 24.94, MAE: 32.00
**************************************************
Ticker: CCI, MAPE: 7.92, MAE: 8.13
**************************************************
Ticker: CPT, MAPE: 14.60, MAE: 16.99
**************************************************
Ticker: CSGP, MAPE: 22.64, MAE: 17.13
**************************************************
Ticker: DLR, MAPE: 10.24, MAE: 17.25
**************************************************
Ticker: DOC, MAPE: 10.16, MAE: 2.10
**************************************************
Ticker: EQIX, MAPE: 10.56, MAE: 93.27
**************************************************
Ticker: EQR, MAPE: 8.94, MAE: 6.

In [28]:
np.mean(list(mapes_arima.values())), np.mean(list(maes_arima.values()))

(12.991628078432619, 18.336621129127455)